# ***Mount Google Drive***



In [44]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
#Install rake_nltk
!pip install rake_nltk

In [46]:
#Import data
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
path = '/content/drive/My Drive/Colab Notebooks/lovelace bootcamp/machine learning part/End project/'
os.listdir(path)
# load the dataset
df = pd.read_csv(path + 'Hotel_Reviews.csv')
df.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [47]:
df['Negative_Review'][0]
type(df['Negative_Review'][0])

str

# Data cleaning

In [0]:
review_summary = pd.DataFrame([])
positive = []
negative = []
hotels = []

for hid in df['Hotel_Name'].unique():
    negative_reviews = df[df['Hotel_Name'] == hid]['Negative_Review']
    positive_reviews = df[df['Hotel_Name'] == hid]['Positive_Review']
    neg_cor = ''.join(negative_reviews).replace(',','')
    pos_cor = ''.join(positive_reviews).replace(',','')
    
    positive.append(pos_cor)
    negative.append(neg_cor)
    hotels.append(hid)

review_summary['Hotel_name'] = hotels
review_summary['Positive_summary'] = positive
review_summary['Negative_summary'] = negative

In [49]:
type(pos_cor)

str

In [50]:
print(str(review_summary[review_summary['Hotel_name']==hid]['Negative_summary']))

1491     Initial receptionist very functional told us ...
Name: Negative_summary, dtype: object


In [0]:
#initializing the new column for negative reviews
review_summary['neg_words'] = ""

for index, row in review_summary.iterrows():
    negative_review = row['Negative_summary']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake(min_length=2, max_length=3)

    # extracting the words by passing the text
    r.extract_keywords_from_text(negative_review)

    # getting the dictionary whith key words as keys and their scores as values
    top_neg = r.get_ranked_phrases()
    
    # assigning the key words to the new column for the corresponding movie
    row['neg_words'] = str(top_neg)
    

In [0]:
#initializing the new column for positive reviews
review_summary['pos_words'] = ""

for index, row in review_summary.iterrows():
    positive_review = row['Positive_summary']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake(min_length=2, max_length=3)

    # extracting the words by passing the text
    r.extract_keywords_from_text(positive_review)

    # getting the dictionary whith key words as keys and their scores as values
    top_pos = r.get_ranked_phrases()
    
    # assigning the key words to the new column for the corresponding movie
    row['pos_words'] = str(top_pos)

In [87]:
top_pos
review_summary.head()

,Hotel_name,Positive_summary,Negative_summary,neg_words,pos_words
0,Hotel Arena,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,"['young men tended', 'young gentlemen managing...","['special mood combined', 'similar properties ..."
1,K K Hotel George,Very comfortable beds smart bathroom good sho...,Really nothing negative to say about this hot...,"['specific questions additionally', 'spare bla...","['working wi fi', 'site manager extended', 'or..."
2,Apex Temple Court Hotel,They have followed through special requests C...,Difficult to locate the phone to contact rece...,"['wracked street corner', 'weekends temple see...","['week days due', 'watch lion king', 'ten adul..."
3,The Park Grand London Paddington,The size of the room Nothing special The fron...,The first night we were given an extremely sm...,"['welcome guys try', 'storing buggy ect', 'sol...","['worst nightmare disgusting', 'wooden panels ..."
4,Monhotel Lounge SPA,Nice hotel Room was beautiful and bed very co...,No Negative Not only did the staff on arrival ...,"['talking non stop', 'quiet moment maybe', 'le...","['wonderfull since reservation', 'window wide ..."


In [54]:
type(review_summary['Negative_summary'])

pandas.core.series.Series

In [0]:
# Based on negative reviews
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(review_summary['neg_words'])

# generating the cosine similarity matrix
cosine_sim_neg = cosine_similarity(count_matrix, count_matrix)

In [0]:
# Based on positive reviews
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(review_summary['pos_words'])

# generating the cosine similarity matrix
cosine_sim_pos = cosine_similarity(count_matrix, count_matrix)

In [0]:
hotels

In [89]:
dtt = pd.DataFrame(cosine_sim_pos, columns = hotels, index = hotels)
dtt.head()


,Hotel Arena,K K Hotel George,Apex Temple Court Hotel,The Park Grand London Paddington,Monhotel Lounge SPA,Kube Hotel Ice Bar,The Principal London,Park Plaza County Hall London,One Aldwych,Splendid Etoile,Hotel Trianon Rive Gauche,InterContinental London Park Lane,Novotel Suites Paris Nord 18 me,Grand Royale London Hyde Park,Milestone Hotel Kensington,Crowne Plaza London Kings Cross,Select Hotel,H tel De Vend me,Pullman Paris Centre Bercy,Gardette Park Hotel,Hotel Panache,H tel des Ducs D Anjou,Hotel Scribe Paris Opera by Sofitel,Novotel London West,Haymarket Hotel,Intercontinental London The O2,H tel Juliana Paris,Hotel L Antoine,Belfast,Shangri La Hotel Paris,Mercure Paris Tour Eiffel Pont Mirabeau,Knightsbridge Hotel,The London EDITION,The Cranley Hotel,Hotel Le 10 BIS,The Nadler Soho,Grange St Paul s Hotel,The Ampersand Hotel,Covent Garden Hotel,The Nadler Victoria,...,Atahotel Linea Uno,Best Western Hotel Astoria,Sheraton Diana Majestic,Hotel degli Arcimboldi,Novotel Milano Nord Ca Granda,Best Western Hotel St George,NH Carlton Amsterdam,Albus Hotel Amsterdam City Centre,Hotel Vilamar,Vilana Hotel,Acevi Villarroel,Hotel Arkadenhof,Hotel Vondel Amsterdam,Hotel Pension Baron am Schottentor,Hotel Atlanta,Park Inn by Radisson Uno City Vienna,NH Danube City,Strandhotel Alte Donau,ARCOTEL Kaiserwasser Superior,Mercure Hotel Raphael Wien,Arthotel ANA Boutique Six,Austria Trend Hotel Anatol Wien,Hotel Kaiserin Elisabeth,Boutique Hotel Notting Hill,Crowne Plaza London Ealing,Good Hotel London,Park Plaza Westminster Bridge London,Hotel du Vin Cannizaro House Wimbledon,Amsterdam Canal Residence,Thistle Trafalgar Square The Royal Trafalgar,Corinthia Hotel London,The Student Hotel Amsterdam City,Hotel Prinz Eugen,Hotel Erzherzog Rainer,Das Triest Hotel,Suite Hotel 900 m zur Oper,Hotel Amadeus,The Berkeley,Holiday Inn London Kensington,Atlantis Hotel Vienna
Hotel Arena,1.000000,0.789815,0.824401,0.807486,0.664465,0.568819,0.818560,0.792323,0.689834,0.686053,0.760084,0.586783,0.732530,0.847966,0.675012,0.819149,0.786559,0.613372,0.745527,0.779172,0.677203,0.685967,0.697337,0.795773,0.680864,0.794798,0.734978,0.744223,0.632228,0.398639,0.618596,0.748573,0.705759,0.753237,0.592436,0.748103,0.805765,0.792630,0.692060,0.789249,...,0.636085,0.585841,0.628135,0.785991,0.573068,0.730756,0.763597,0.769109,0.720078,0.723658,0.771597,0.465550,0.819535,0.745361,0.762446,0.718690,0.708642,0.617989,0.626731,0.652305,0.715495,0.737199,0.747717,0.880602,0.743563,0.809966,0.806864,0.790259,0.701263,0.700703,0.726892,0.862298,0.703215,0.751812,0.550332,0.731174,0.658983,0.640852,0.814516,0.728668
K K Hotel George,0.789815,1.000000,0.866450,0.908257,0.649313,0.479955,0.885068,0.863287,0.703345,0.710546,0.841472,0.683035,0.725881,0.907874,0.626575,0.842005,0.836080,0.734943,0.815596,0.780477,0.688180,0.803064,0.765038,0.904051,0.720139,0.802540,0.802953,0.780889,0.708957,0.376460,0.677698,0.770950,0.664054,0.752240,0.593391,0.813885,0.852308,0.817877,0.643202,0.835684,...,0.710247,0.636315,0.684869,0.853670,0.635683,0.836508,0.860678,0.778029,0.823675,0.714983,0.843171,0.465042,0.869701,0.783279,0.807745,0.816249,0.795884,0.706288,0.713528,0.723561,0.863902,0.806837,0.850557,0.832976,0.836303,0.831627,0.869215,0.828685,0.714263,0.837135,0.711353,0.793754,0.828157,0.867102,0.512366,0.843971,0.799458,0.650061,0.907336,0.820421
Apex Temple Court Hotel,0.824401,0.866450,1.000000,0.909565,0.698011,0.595700,0.911751,0.922905,0.779844,0.787796,0.820675,0.681610,0.752698,0.906327,0.689672,0.892250,0.838699,0.760009,0.775012,0.754246,0.699759,0.735913,0.815166,0.898706,0.792844,0.872928,0.816289,0.807366,0.716014,0.426362,0.689890,0.847172,0.751107,0.814002,0.615794,0.888128,0.902208,0.885837,0.723976,0.902201,...,0.677420,0.601330,0.633660,0.769142,0.614566,0.745052,0.857255,0.867324,0.780030,0.728564,0.843414,0.387274,0.878341,0.806286,0.764756,0.739944,0.728754,0.638082,0.623448,0.671335,0.755399,0.769919,0.853518,0.897929,0.841416,0.864195,0.929351,0.846145,0.

In [90]:
dtt['Hotel Arena'].sort_values(ascending = False)

Hotel Arena                                  1.000000
WestCord Fashion Hotel Amsterdam             0.891439
Boutique Hotel Notting Hill                  0.880602
Corendon Vitality Hotel Amsterdam            0.872163
Golden Tulip Amsterdam West                  0.872148
Ramada Apollo Amsterdam Centre               0.871169
The Student Hotel Amsterdam City             0.862298
Golden Tulip Amsterdam Riverside             0.860652
INK Hotel Amsterdam MGallery by Sofitel      0.860489
Hotel De Hallen                              0.859634
Mercure Hotel Amsterdam City South           0.858255
Park Plaza Vondelpark Amsterdam              0.858066
Bilderberg Garden Hotel                      0.857742
Room Mate Aitana                             0.856250
Apollo Hotel Amsterdam                       0.854421
Hampshire Hotel The Manor Amsterdam          0.853519
citizenM Amsterdam                           0.848271
Grand Royale London Hyde Park                0.847966
Clayton Crown Hotel London  

In [88]:
dtt = pd.DataFrame(cosine_sim_neg, columns = hotels, index = hotels)
dtt.head()
dtt['Hotel Arena'].sort_values(ascending = False)

Hotel Arena                                               1.000000
Park Plaza Victoria Amsterdam                             0.879016
Hampshire Hotel Amsterdam American                        0.871207
Park Plaza London Riverbank                               0.870486
The Cumberland A Guoman Hotel                             0.863906
Park Plaza Vondelpark Amsterdam                           0.863537
Millennium Gloucester Hotel London                        0.863034
NH Carlton Amsterdam                                      0.862380
Doubletree by Hilton London Kensington                    0.862111
Hilton London Metropole                                   0.860330
TheWesley                                                 0.860145
The Park Grand London Paddington                          0.859384
St James Court A Taj Hotel London                         0.859049
The Principal London                                      0.858785
NH Collection Amsterdam Grand Hotel Krasnapolsky          0.85

In [0]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(review_summary.index)

# idx = review_summary['Hotel_name' == 'Hotel_Arena'].get_loc()

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(hotel_picked, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_hotels = []
    
    # gettin the index of the hotel that matches the title
    idx = indices[indices == hotel_picked].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(df.index)[i])
        
    return recommended_hotels

In [73]:
indices[0]
#Top 10 recommendations based on the negatiev reviews
# recommendations('Hotel_Arena',cosine_sim = cosine_sim_neg)

0

In [0]:
#Top 10 recommendations based on the negatiev reviews